## VRE Showcase

This notebook is also hosted in Google Colab: [link](https://colab.research.google.com/drive/1vAp71H-TLewhF56odv33TkmGwwhuoFJ-?usp=sharing#scrollTo=xKjeYBfhJrJl)

Provides and end-to-end run of VRE from a single video, to computing a few learned representations (aka experts) and
then getting derived representations using the "semantic_mapper" module which only combines multiple pre-extracted
representations w/o any neural networks, only simple arithmetics.

In [ ]:
# !pip install video-representations-extractor==1.12.2 -U --no-cache
# !ls /usr/local/lib/python3.11/dist-packages/vre_repository/semantic_segmentation/mask2former/mask2former_impl/resources

In [ ]:
from pathlib import Path
import os
import urllib
os.environ["VRE_LOGLEVEL"] = "0"
from pprint import pprint
import subprocess
import numpy as np
import torch as tr
import matplotlib.pyplot as plt
import subprocess
import shutil

from vre.representations import build_representations_from_cfg, ReprOut, Representation
from vre.readers.multitask_dataset import MultiTaskDataset, MultiTaskItem
from vre.utils import collage_fn, image_add_title, MemoryData
from vre_repository import get_vre_repository

from pathlib import Path
import urllib
import sys
sys.path.append("examples/semantic_mapper")
urllib.request.urlretrieve("https://gitlab.com/video-representations-extractor/video-representations-extractor/-/raw/master/examples/semantic_mapper/semantic_mapper.py", filename="semantic_mapper.py")
urllib.request.urlretrieve("https://gitlab.com/video-representations-extractor/video-representations-extractor/-/raw/master/examples/semantic_mapper/cfg.yaml", filename="cfg.yaml")
from semantic_mapper import get_new_semantic_mapped_tasks

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
video_path = Path("test_video.mp4")
video_url = "https://gitlab.com/video-representations-extractor/video-representations-extractor/-/raw/master/resources/test_video.mp4"
if True: # Chance this if you don't want to download the movie every time
    shutil.rmtree(video_path, ignore_errors=True)
if not Path(video_path).exists(): # download a new one and change it to your liking
    urllib.request.urlretrieve(video_url, filename=str(video_path))

cfg_path = Path("cfg.yaml")
vre_dir = Path(f"data_{video_path.name}")
# frames = ["1", "100", "1000", "3000"]
# frames = [str(x) for x in np.random.randint(0, 5000, size=(3, ))]
frames = ["3420", "3891", "1947"]
vre_device = "cuda" if tr.cuda.is_available() else "cpu"
print(vre_device)
# if vre_dir.exists():
#     shutil.rmtree(vre_dir)
args = ["vre", str(video_path), "--config_path", str(cfg_path), "-o", str(vre_dir),
        "--frames", *frames, "--output_dir_exists_mode", "skip_computed"]
print(f"Running: {' '.join(args)}")
res = subprocess.run(args=args, env={**os.environ.copy(), **{"VRE_DEVICE": vre_device}}, capture_output=True, text=True)
if res.returncode != 0:
    print("============")
    print(res.stderr)
    print("============")
    print(res.stderr)


In [ ]:
def plot_one(data: MultiTaskItem, title: str, order: list[str],
             name_to_task: dict[str, Representation]) -> np.ndarray:
    def vre_plot_fn(rgb: tr.Tensor, x: tr.Tensor, node: Representation) -> np.ndarray:
        res = ReprOut(rgb.cpu().detach().numpy()[None], MemoryData(x.cpu().detach().numpy()[None]), [0])
        return node.make_images(res)[0]
    order = [*[k for k in data if k in order], *[k for k in data if k not in order]]
    img_data = {k: vre_plot_fn(data["rgb"].byte(), data[k], name_to_task[k]) for k in order}
    titles = [title if len(title) < 40 else f"{title[0:19]}..{title[-19:]}" for title in img_data]
    collage = collage_fn(list(img_data.values()), titles=titles, size_px=40)
    collage = image_add_title(collage, title, size_px=55, top_padding=110)
    return collage

task_names = ["rgb", "depth_marigold", "normals_svd(depth_marigold)", "opticalflow_rife",
              "semantic_mask2former_coco_47429163_0", "semantic_mask2former_mapillary_49189528_0"]
order = ["rgb", "semantic_mask2former_mapillary_49189528_0", "semantic_mask2former_coco_47429163_0",
         "depth_marigold", "normals_svd(depth_marigold)"]

task_types = {r.name: r for r in build_representations_from_cfg(cfg_path, get_vre_repository()) if r.name in task_names}

reader = MultiTaskDataset(vre_dir, task_names=task_names, task_types=task_types, handle_missing_data="fill_nan",
                          normalization="min_max", cache_task_stats=True, batch_size_stats=100)
orig_task_names = list(reader.task_types.keys())

print(reader)
print("== Shapes ==")
pprint(reader.data_shape)

data, name = reader[np.random.randint(0, len(reader))]
collage = plot_one(data, title=name, order=order, name_to_task=reader.name_to_task)
plt.figure(figsize=(15, 5))
plt.imshow(collage)
plt.show()

In [ ]:
new_tasks = get_new_semantic_mapped_tasks(include_semantic_output=False)
for task_name in reader.task_names:
    if task_name not in orig_task_names and task_name in reader.task_names:
        reader.remove_task(task_name)
for new_task in new_tasks.values():
    reader.add_task(new_task, overwrite=True)

print("== Random loaded item ==")
ixs = np.random.permutation(range(len(reader))).tolist()
for ix in ixs:
    data, name = reader[ix] # get a random item
    plt.figure(figsize=(20, 10))
    plt.imshow(collage := plot_one(data, title=name, order=order, name_to_task=reader.name_to_task))
    plt.show()
    break